In [20]:
import ee
import geemap
import os
import ipyleaflet
import ipywidgets as widgets
import math
from ipyleaflet import GeoJSON, Marker, MarkerCluster
from datetime import datetime
import psycopg2
from psycopg2 import Error

##############################
#Задачи:
# исправить литологию [x]
# палитру для площадной вероятности взять из USGS [x]
# реализовать доступ к API eq alert [x]
# реализовать экспорт [x]
# заменить basemap [x]
# создание маркера при клике на кнопку [x]

################################

#Создаём объект карты для отображения будущих данных
Map = geemap.Map(toolbar_ctrl=False,layer_ctrl=True, basemap="HYBRID")
Map.layout.height = '710px'

#  Класс для объектов с типом событий землетрясений
# используется для хранения данных импортированных из БД
class Eq_Event:
  def __init__(self, lat, lon, depth, mag):
    self.lat = lat
    self.lon = lon
    self.depth = depth
    self.mag = mag

#Палитра цветов для отображения вероятностей сейсмогенных оползней:
landslide_palette=['#FFFFFF','#FFFFFF',' #F3F3C2','#F3F3C2',' #F3F3C2',' #F3F3C2',' #F3F3C2',' #F3F3C2',' #F3F3C2',' #F3F3C2','#EBD258','#EBD258','#EBD258','#EBD258','#EBD258','#EBD258','#EBD258','#EBD258','#EBD258','#EBD258','#EF8F39','#EF8F39','#EF8F39','#EF8F39','#EF8F39','#EF8F39','#EF8F39','#EF8F39','#EF8F39','#EF8F39','#EF8F39','#EF8F39','#EF8F39','#EF8F39','#EF8F39','#EF8F39','#EF8F39','#EF8F39','#EF8F39','#EF8F39','#EF8F39','#EF8F39','#EF8F39','#EF8F39','#EF8F39','#EF8F39','#EF8F39','#EF8F39','#EF8F39','#EF8F39','#CC607D','#CC607D','#CC607D','#CC607D','#CC607D','#CC607D','#CC607D','#CC607D','#CC607D','#CC607D','#CC607D','#CC607D','#CC607D','#CC607D','#CC607D','#CC607D','#CC607D','#CC607D','#CC607D','#CC607D','#CC607D','#CC607D','#CC607D','#CC607D','#CC607D','#CC607D','#CC607D','#CC607D','#CC607D','#CC607D','#CC607D','#CC607D','#CC607D','#CC607D','#CC607D','#CC607D','#CC607D','#CC607D','#CC607D','#CC607D','#CC607D','#CC607D','#CC607D','#CC607D','#CC607D','#CC607D','#CC607D','#CC607D','#CC607D','#CC607D','#7D54C2','#7D54C2','#7D54C2','#7D54C2','#7D54C2','#7D54C2','#7D54C2','#7D54C2','#7D54C2','#7D54C2','#7D54C2','#7D54C2','#7D54C2','#7D54C2','#7D54C2','#7D54C2','#7D54C2','#7D54C2','#7D54C2','#7D54C2','#7D54C2','#7D54C2','#7D54C2','#7D54C2','#7D54C2','#7D54C2','#7D54C2','#7D54C2','#7D54C2','#7D54C2','#7D54C2','#7D54C2','#7D54C2','#7D54C2','#7D54C2','#7D54C2','#7D54C2','#7D54C2','#7D54C2','#7D54C2','#7D54C2','#7D54C2','#7D54C2','#7D54C2','#7D54C2','#7D54C2','#7D54C2','#7D54C2','#7D54C2','#7D54C2','#7D54C2','#7D54C2','#7D54C2','#7D54C2','#7D54C2','#7D54C2','#7D54C2','#7D54C2','#7D54C2','#7D54C2','#7D54C2','#7D54C2','#7D54C2','#7D54C2','#7D54C2','#7D54C2','#7D54C2','#7D54C2','#7D54C2','#7D54C2','#7D54C2','#7D54C2','#7D54C2','#7D54C2','#7D54C2','#7D54C2','#7D54C2','#7D54C2','#7D54C2','#7D54C2','#7D54C2','#7D54C2','#7D54C2','#7D54C2','#7D54C2','#7D54C2','#7D54C2','#7D54C2','#7D54C2','#7D54C2','#7D54C2','#7D54C2','#7D54C2','#7D54C2','#7D54C2','#7D54C2','#7D54C2','#7D54C2','#7D54C2','#7D54C2','#4C4C82','#4C4C82','#4C4C82','#4C4C82','#4C4C82','#4C4C82','#4C4C82','#4C4C82','#4C4C82','#4C4C82','#4C4C82','#4C4C82','#4C4C82','#4C4C82','#4C4C82','#4C4C82','#4C4C82','#4C4C82','#4C4C82','#4C4C82','#4C4C82','#4C4C82','#4C4C82','#4C4C82','#4C4C82','#4C4C82','#4C4C82','#4C4C82','#4C4C82','#4C4C82','#4C4C82','#4C4C82','#4C4C82','#4C4C82','#4C4C82','#4C4C82','#4C4C82','#4C4C82','#4C4C82','#4C4C82','#4C4C82','#4C4C82','#4C4C82','#4C4C82','#4C4C82','#4C4C82','#4C4C82','#4C4C82','#4C4C82','#4C4C82','#4C4C82','#4C4C82','#4C4C82','#4C4C82','#4C4C82','#4C4C82','#4C4C82','#4C4C82','#4C4C82','#4C4C82','#4C4C82','#4C4C82','#4C4C82','#4C4C82','#4C4C82','#4C4C82','#4C4C82','#4C4C82','#4C4C82','#4C4C82','#4C4C82','#4C4C82','#4C4C82','#4C4C82','#4C4C82','#4C4C82','#4C4C82','#4C4C82','#4C4C82','#4C4C82','#4C4C82','#4C4C82','#4C4C82','#4C4C82','#4C4C82','#4C4C82','#4C4C82','#4C4C82','#4C4C82','#4C4C82','#4C4C82','#4C4C82','#4C4C82','#4C4C82','#4C4C82','#4C4C82','#4C4C82','#4C4C82','#4C4C82','#4C4C82']

###    Загрузка наборов данных:

##Импорт данных Vs30 - USGS_Vs30
USGS_Vs30 = ee.Image("users/iOne/USGS_Vs30").select('b1')

##Импорт данных высот - SRTM90_V4
SRTM = ee.Image('CGIAR/SRTM90_V4').select('elevation')

##Импорт данных о типе поверхности - GlobCover
land = ee.Image('ESA/GLOBCOVER_L4_200901_200912_V2_3')
LandCoverSelect = land.select('landcover')
#Замена значений типа поверхности на соответствующий коэффициент вероятностной модели
LandCoverToCoeffs = land.remap([14,20,30,40,50,60,70,90,100,110,120,130,140,150,160,180,190,200,220,230],[0.91,0.88,0.78,0.68,0.30,1.77,1.71,-1.26,1.50,0.68,1.13,0.79,1.03,0.54,2.34,1.19,0.30,-0.06,-0.18,-1.08],-1.08)

##Импорт данных о литологии
lithol = ee.Image("users/iOne/litho")
lith_val = lithol.select('b2')
#Замена значений типа литологии на соответствующий коэффициент вероятностной модели
LithologyToCoeffs = lith_val.remap([85,47,15,72,127,255,192,185,148,191],[-1.87,-0.78,-1.88,-1.61,-1.92,-1.92,-3.22,-1.50,-0.81,-3.22],-1.92)

##Импорт данных о суммарных стоках - HydroSHEDS
dataset = ee.Image('WWF/HydroSHEDS/15ACC')
flowAccumulation = dataset.select('b1')

#Вычисляем на основе данных высот значения углов склонов
slope = ee.Terrain.slope(SRTM)
slope_change = slope.expression('b(0) <= 0 ? 1 : b(0)')

#Конвертируем градусы в радианы
slope_to_rad = slope_change.divide(180).multiply(math.pi)
#Вычисляем тангенс углов склона
tan_rad_slope = slope_to_rad.tan()
#  К величине суммарного стока добавляем единицу и умножаем на размер сетки
#данных о высот. В нашем случае размер сетки равен 90 метров.
fa_s = flowAccumulation.add(1).multiply(90)

#Вычисляем Compound Topographic Index
# CTI = ln(Fa/tg(a)) , где Fa- это величина суммарного стока приведенная к раземеру сетки, a - угол наклона склона в радианах
CTI = fa_s.divide(tan_rad_slope).log()

#Многоугольник ограничивающий зону острову Сахалин
SakhArea=ee.Geometry.Polygon([[140.6718, 45.7835],[141.5836, 52.1947],[141.6825, 54.4961], [144.6927, 54.5535],[144.9564, 48.6260],[143.9017,45.7221],[140.6718, 45.7835]])

#Координаты по умолчанию
CenterLong = 142.73
CenterLat = 46.95

#Слой с координатами всех точек на карте.
LongLat = ee.Image.pixelLonLat()
lat = LongLat.select('latitude')
lon = LongLat.select('longitude')

# Функция для создания слоя со значениями гипоцентрального расстояния
def haversineFunction(imageLat, imageLong, point_lat, point_lon, depth):
  fi1 = ee.Image(ee.Number(point_lat)).divide(180).multiply(math.pi)
  fi2 = imageLat.divide(180).multiply(math.pi)
  lambda1 = ee.Image(ee.Number(point_lon)).divide(180).multiply(math.pi)
  lambda2 = imageLong.divide(180).multiply(math.pi)
  delta_fi = fi2.subtract(fi1)
  delta_lambda = lambda2.subtract(lambda1)
  p1 = delta_fi.divide(2).sin().multiply(delta_fi.divide(2).sin())
  p2 = fi1.cos().multiply(fi2.cos())
  p3 = delta_lambda.divide(2).sin().multiply(delta_lambda.divide(2).sin())
  a = p2.multiply(p3).add(p1)
  p4 = ee.Image(ee.Number(1)) 
  p5 = ee.Image(ee.Number(2)) 
  p6 = p4.subtract(a).sqrt()
  p7 = a.sqrt()
  p8 = p6.atan2(p7)
  c = p5.multiply(p8)
  R = ee.Image(ee.Number(6371)) 
  d = R.multiply(c)

  hypocentral = d.pow(2).add(depth*depth).sqrt()
  return hypocentral

#Функция для создания слоя пиковых скоростей грунта на основе закона затухания
def damping_law_Function(image_X, Mw):
  a2 = 0.8431
  b2_I = -0.00287
  c2_I = -2.8590
  d2 = 0.002109
  e2 = 0.5
  #sigma2 = 0.341184
  sigma2 = 0.2645
  ps = -0.5546
  Vsmax =  1100
  V0 = 350
  Mw02 = 8.1

  Mw2 = min(Mw,Mw02)
  LogHelp = ee.Image(ee.Number(1)).expression(
     'X + (d2 * 10 ** (e2 * Mw2))',{
      'X': image_X.select('constant'),
      'd2':d2 ,
      'e2':e2 ,
      'Mw2':Mw2
      })
  LogHelp = LogHelp.log10()

  LogA = ee.Image(ee.Number(1)).expression(
    'A = a2 * Mw2 + b2_I * X + c2_I - LogX',{
    'X': image_X.select('constant'),
    'LogX': LogHelp.select('constant'),
    'a2':a2 ,
    'Mw2':Mw2 ,
    'b2_I':b2_I ,
    'c2_I':c2_I
    })

  Gs = USGS_Vs30.min(ee.Number(Vsmax)).divide(V0).log10().multiply(ps)

  LogAGs = LogA.add(Gs)

  A = ee.Image(ee.Number(1)).expression(
  'A = 10**LogAGs',{
  'LogAGs': LogAGs.select('A')
  })
  LnA = A.log()
  return LnA

#Функция для создания слоя площадных вероятностей сейсмогенных оползней
def LandSlideProbability(pLat,pLong,dep,magn):
 distance = haversineFunction(lat,lon,pLat,pLong,dep)
 LnPGV = damping_law_Function(distance, magn)
 Landslide = slope.expression(
    '1/(1+exp**(-(-6.3 + 1.65 * lnPGV + 0.06 * slope + lithology + landcover + 0.03 * cti + 0.01 * lnPGV * slope)))', {
      'slope': slope.select('slope'),
      'exp':math.exp(1),
      'lnPGV':LnPGV.select('A'),
      'landcover':LandCoverToCoeffs.select('remapped'),
      'lithology':LithologyToCoeffs.select('remapped'),
      'cti': CTI.select('b1')
 })
 #Map.addLayer(Landslide.clip(SakhArea),{'min': 0, 'max' :1,'palette':['#008608','#E7FF06','#EC8300','#FF2806']},'Somes');
 LandslideSqare = slope.expression(
      'constant=exp**(-a+b*p-c*(p**2)+d*(p**3))', {
      'p':Landslide.select('constant'),
      'exp':math.exp(1),
      'a': 7.592,
      'b': 5.237,
      'c': 3.042,
      'd': 4.035
 })     
 return LandslideSqare


####Описание виджетов
#Блок описывающий элементы меню
output_widget = widgets.Output(layout={'border': '1px solid black'})
output_control = ipyleaflet.WidgetControl(widget=output_widget, position='bottomright')
Map.add_control(output_control)

from ipywidgets import Layout

tabs_label= widgets.HTML(
    value= '<style>p{word-wrap: break-word}</style> <p align="justify">'+ 'Введите значения для расчёта вероятности оползня, координаты эпицентра землетрясения можно задать кликнув по карте' +' </p>',
    layout=Layout(width='250px',height='100px',margin='10px')
)

long_tb =widgets.BoundedFloatText(
            value=142.516,
            layout=Layout(width='250px'),
            min=140.281,
            max=147.111,
            step=0.1,
            description='Долгота:',
            disabled=False)
lat_tb = widgets.BoundedFloatText(
            value=47.246,
            layout=Layout(width='250px'),
            min=45.743,
            max=54.623,
            step=0.1,
            description='Широта:',
            disabled=False)
mag_tb = widgets.BoundedFloatText(
            value=7,
            layout=Layout(width='250px'),
            min=2.5,
            max=10.0,
            step=0.1,            
            description='Магнитуда:',
            disabled=False)
depth_tb = widgets.BoundedFloatText(
            value=7,
            layout=Layout(width='250px'),
            min=1,
            max=1000,
            step=10,
            description='Глубина:',
            disabled=False)
man_but = widgets.Button(
            description='Пуск',
            layout=Layout(width='250px'),
            disabled=False,
            button_style='success', # 'success', 'info', 'warning', 'danger' or ''
            #tooltip='Run',
            icon='check')

manual_vbox = widgets.VBox([tabs_label,long_tb,lat_tb,mag_tb,depth_tb,man_but])

###Блок выбора импортированных данных о землетрясениях
          
earthq_label= widgets.HTML(
    value= '<style>p{word-wrap: break-word}</style> <p align="justify">'+ 'Фильтр' +' </p>',
    layout=Layout(width='250px',height='100px',margin='10px')
)
##Фильтр по магнитуде
mag_name = widgets.Label(value='Магнитуда')
mag_tb_min = widgets.BoundedFloatText(
            value=2.5,
            layout=Layout(width='100px'),
            min=2.5,
            max=10.0,
            step=0.1,            
            #description='Магнитуда:',
            disabled=False)
mag_betw = widgets.Label(value='-')
mag_tb_max = widgets.BoundedFloatText(
            value=10.0,
            layout=Layout(width='100px'),
            min=2.5,
            max=10.0,
            step=0.1,            
            #description='Магнитуда:',
            disabled=False)

mags = widgets.HBox([mag_tb_min,mag_betw,mag_tb_max])

##Фильтр по широте
lat_name = widgets.Label(value='Широта')
lat_tb_min = widgets.BoundedFloatText(
            value=45.743,
            layout=Layout(width='100px'),
            min=45.743,
            max=54.623,
            step=0.1,
            #description='Широта:',
            disabled=False)
lat_betw = widgets.Label(value='-')
lat_tb_max = widgets.BoundedFloatText(
            value=54.623,
            layout=Layout(width='100px'),
            min=45.743,
            max=54.623,
            step=0.1,
            #description='Широта:',
            disabled=False)

lats = widgets.HBox([lat_tb_min,lat_betw,lat_tb_max])

##Фильтр по долготе
lon_name = widgets.Label(value='Долгота')
lon_tb_min =widgets.BoundedFloatText(
            value=140.281,
            layout=Layout(width='100px'),
            min=140.281,
            max=147.111,
            step=0.1,
            #description='Долгота:',
            disabled=False)
lon_betw = widgets.Label(value='-')
lon_tb_max =widgets.BoundedFloatText(
            value=147.111,
            layout=Layout(width='100px'),
            min=140.281,
            max=147.111,
            step=0.1,
            #description='Долгота:',
            disabled=False)

lons = widgets.HBox([lon_tb_min,lon_betw,lon_tb_max])

##Фильтр по глубине
dep_name = widgets.Label(value='Глубина (км)')
dep_tb_min = widgets.BoundedFloatText(
            value=1,
            layout=Layout(width='100px'),
            min=1,
            max=1000,
            step=5,
            #description='Глубина (км):',
            disabled=False)
dep_betw = widgets.Label(value='-')
dep_tb_max = widgets.BoundedFloatText(
            value=50,
            layout=Layout(width='100px'),
            min=1,
            max=1000,
            step=5,
            #description='Глубина (км):',
            disabled=False)

deps = widgets.HBox([dep_tb_min,dep_betw,dep_tb_max])

##Фильтр по дате
date_name = widgets.Label(value='Дата и время')
date_min_pick = widgets.DatePicker(            
            layout=Layout(width='170px'),  
            value = datetime.now(),
            #description='C:',
            disabled=False)
date_min_lab = widgets.Label(value='C: ', layout=Layout(width='30px'))
#date_betw = widgets.Label(value='-')
date_max_pick = widgets.DatePicker(            
            layout=Layout(width='170px'), 
            value = datetime.now(),
            #description='По:',
            disabled=False)
date_max_lab = widgets.Label(value='По: ', layout=Layout(width='30px'))

date_min = widgets.HBox([date_min_lab,date_min_pick])
date_max = widgets.HBox([date_max_lab,date_max_pick])
##Кнопка для применения фильтра
filter_but = widgets.Button(
            description='Применить фильтр',
            layout=Layout(width='205px'),
            disabled=False,
            button_style='success', # 'success', 'info', 'warning', 'danger' or ''
            #tooltip='Run',
            icon='check')

####Группировка виджетов фильтров
filter_vbox = widgets.VBox([mag_name, mags,lat_name,lats,lon_name,lons,dep_name,deps,date_name,date_min,date_max,filter_but])   

filter_eq = widgets.Accordion([filter_vbox]) 
filter_eq.set_title(0, 'Фильтр событий')

## Блок виджетов для импорта данных
import_start = widgets.Label(value='Выберите событие из списка:')

import_eq_select = widgets.Select(disabled=False,
                                  layout=Layout(width='250px',height='150px'),
                                  options=['Примените фильтр'])

import_cur_lab = widgets.Label(value='Выбранное событие:')

import_cur_select = widgets.Textarea(
    layout=Layout(width='250px', height='50px'),
    placeholder='Выбранное событие',
    disabled=True
)

import_but = widgets.Button(
            description='Выбрать событие',
            layout=Layout(width='250px'),
            disabled=True,
            button_style='success', # 'success', 'info', 'warning', 'danger' or ''
            #tooltip='Run',
            icon='check')

import_vbox = widgets.VBox([import_start,import_eq_select,import_cur_lab,import_cur_select,import_but])

import_eq = widgets.Accordion([import_vbox],selected_index=None)
import_eq.set_title(0, 'Выбор события')

earthq_vbox = widgets.VBox([filter_eq, import_eq])   

###Блок экспорта данных в формате GeoTIFF
export_label= widgets.HTML(
    value= '<style>p{word-wrap: break-word}</style> <p align="justify">'+ 'Экспорт данных в формате GeoTiff. Предварительно выполните вычисления нажав клавишу "Пуск" на вкладке "Ручной ввод"' +' </p>',
    layout=Layout(width='250px',height='110px',margin='10px')
)

exp_but = widgets.Button(
            description='Экспорт',
            layout=Layout(width='250px'),
            disabled=True,
            button_style='success', # 'success', 'info', 'warning', 'danger' or ''
            #tooltip='Run',
            icon='check')
export_link= widgets.HTML(
    value= '<style>p{word-wrap: break-word}</style><p align="justify">Ссылка на скачивание: </p> <p align="justify">'+ 'Сначала выполните экспорт' +' </p>',
    layout=Layout(width='250px',height='100px',margin='10px')
)
export_vbox = widgets.VBox([export_label,exp_but,export_link])  

######

tab = widgets.Tab([manual_vbox,earthq_vbox,export_vbox])
tab.set_title(0, 'Ручной ввод')
tab.set_title(1, 'События')
tab.set_title(2, 'Экспорт')

menu = widgets.Accordion(children=[tab],selected_index=None)
menu.set_title(0, 'Меню')
with output_widget:
 display(menu)

#Конец блока меню

#Виджет для отображения вероятности сейсмогенного оползня в точке
Probability_widget = widgets.Output(layout={'border': '1px solid black'})
Probability_control = ipyleaflet.WidgetControl(widget=Probability_widget, position='bottomleft')
Map.add_control(Probability_control)

prob_lab = widgets.Label(value='Вероятность:', layout=Layout(width='90px'))

prob_text=widgets.Text(
    value='0%',
    placeholder='Кликните на карте',
    #description='Вероятность:',
    layout=Layout(width='90px'),
    disabled=True
)

prob_vbox = widgets.VBox([prob_lab,prob_text])

with Probability_widget:
 display(prob_vbox)
####Конец описания виджетов

# Функция события нажатия кнопки "Пуск", для вычисления площадных вероятностей сейсмогенных оползней
def but_click_Landslide(a):
 Map.clear_layers()
 Map.add_basemap('HYBRID')
 cur_Lat=lat_tb.value
 cur_Long=long_tb.value
 cur_Depth=depth_tb.value
 cur_Mag=mag_tb.value
 global Opolzn
 Opolzn = LandSlideProbability(cur_Lat,cur_Long,cur_Depth,cur_Mag)
 Opolzn = Opolzn.updateMask(Opolzn.gte(0.001))
 global LandSlideCopy
 LandSlideCopy = Opolzn
 exp_but.disabled=False
 Map.addLayer(Opolzn.clip(SakhArea),{'min': 0, 'max':0.3, 'palette':landslide_palette},'Оползни')
 Map.setCenter(cur_Long,cur_Lat)
 center = (cur_Lat, cur_Long)
 marker = Marker(location=center, draggable=False, title='Эпицентр',name='Эпицентр')
 Map.add_layer(marker)

#Назначение кнопке обработчика события при нажатии
man_but.on_click(but_click_Landslide)

#Функция экспорта данных вероятностей оползней в формате GeoTIFF
def but_click_Export(a):
 link = LandSlideCopy.getDownloadURL({
 'name': datetime.today().strftime('%Y-%m-%d %H-%M-%S'),   
 'scale': 850,
 'crs': 'EPSG:4326',
 'fileFormat': 'GeoTIFF',
 'region': SakhArea})
 export_link.value= '<style>p{word-wrap: break-word}</style><p align="justify">Ссылка на скачивание: </p> <p align="justify">'+ '<a target="_blank" href=\"'+link+'\">Cкачать</a>' +' </p>'

#Назначение кнопке обработчика события при нажатии
exp_but.on_click(but_click_Export)   

#Функция импорта координат из точки по которой кликнул пользователь
def latlon_interaction(**kwargs):
    latlon = kwargs.get('coordinates')
    if kwargs.get('type') == 'click':
        Map.default_style = {'cursor': 'wait'}
        click_point = ee.Geometry.Point(latlon[::-1])
        try:
            #print(round(latlon[0],3))
            long_tb.value=round(latlon[1],3)
            lat_tb.value=round(latlon[0],3)
            Map.default_style = {'cursor': 'pointer'}
        except Exception as e:            
            a=e
        Map.default_style = {'cursor': 'pointer'}
        
#Назначение карте обработчика события при клике по рабочей области
Map.on_interaction(latlon_interaction)

#Функция импорта данных о значение вероятности сейсмогенного оползня в точке по которой кликнул пользователь
def prob_interaction(**kwargs):    
    latlon = kwargs.get('coordinates')
    if kwargs.get('type') == 'click':
        Map.default_style = {'cursor': 'wait'}
        click_point = ee.Geometry.Point(latlon[::-1])
        try:
            probability = Opolzn.reduceRegion(ee.Reducer.first(), click_point, 90);
            result = probability.get('constant').getInfo();
            temp=str(round(result,4)*100)
            prob_text.value=temp[0:5]+'%'
            #print(round(result,4)*100)
            Map.default_style = {'cursor': 'pointer'}
        except Exception as e:
            prob_text.value=('Нет значений')
        Map.default_style = {'cursor': 'pointer'}
        
#Назначение карте обработчика события при клике по рабочей области
Map.on_interaction(prob_interaction)

#Функция импорта данных о землетрясениях из БД
def but_click_Import(a):
 try:
    connection = psycopg2.connect(user="adminsls",password="****************",host="94.228.115.13",port="5432",database="sakhlandslide")
    cursor = connection.cursor()
    select_query="select * from eqalert where (mag between %s and %s) and (lat between %s and %s) and (lon between %s and %s) and (\"depth\" between %s and %s) and (event_datetime between %s and %s) ORDER BY event_datetime DESC"
    select_options = (mag_tb_min.value,mag_tb_max.value,lat_tb_min.value,lat_tb_max.value,lon_tb_min.value,lon_tb_max.value,dep_tb_min.value,dep_tb_max.value,date_min_pick.value,date_max_pick.value)
    cursor.execute(select_query,select_options)  
    list_of_elems = []
    global list_of_events
    list_of_events = []
    
    for row in cursor:
        temp =row[2].strftime("%d/%m/%Y,%H:%M:%S") + "; Ш: "+ str(row[3])[0:5] +"; Д: "+ str(row[4])[0:5] + "; Г: "+str(row[5])+"; М: "+str(row[6]) + "; "+row[8]
        list_of_elems.append(temp)
        temp_obj = Eq_Event(row[3],row[4],row[5],row[6])
        list_of_events.append(temp_obj)
    
    import_eq_select.options=list_of_elems    
    filter_eq.selected_index=None
    import_eq.selected_index=0
    import_but.disabled = False
 except (Exception, Error) as error:
    mem_error=error
 finally:
    if connection:
        cursor.close()
        connection.close()
    list_of_elems.clear()
  
#Назначение кнопке обработчика события при нажатии
filter_but.on_click(but_click_Import)

def eq_select_click_index(change):
    global index_eq
    index_eq = change.new
import_eq_select.observe(eq_select_click_index,'index')

def eq_select_click_value(change):
    import_cur_select.value=change.new

import_eq_select.observe(eq_select_click_value,'value')

def import_eq_accordition(change):
    filter_eq.selected_index=None
import_eq.observe(import_eq_accordition)

def filter_eq_accordition(change):
    import_eq.selected_index=None
filter_eq.observe(filter_eq_accordition)

#Функция импорта данных о землетрясениях в поля вкладки "Ручной ввод"
def but_import_click (a):
    temp = list_of_events[index_eq]
    long_tb.value = temp.lon
    lat_tb.value = temp.lat
    mag_tb.value = temp.mag
    depth_tb.value = temp.depth
    tab.selected_index=0

#Назначение кнопке обработчика события при нажатии    
import_but.on_click(but_import_click)

#Словарь для создания легенды карты
legend_dict = {
    '< 0.2%':'#FFFFFF',
    '0.2 - 1%':'#F3F3C2',
    '1 - 2%':'#EBD258',
    '2 - 5%':'#EF8F39',
    '5 - 10%':'#CC607D',
    '10 - 20%':'#7D54C2',
    '> 20%':'#4C4C82'
}

#Добавление легенды на карту
Map.add_legend(legend_title="Вер-ть оползня", legend_dict=legend_dict, position='bottomleft')



##For dashboard of maps

#Map.addLayer(ee.Image('WWF/HydroSHEDS/15ACC').clip(SakhArea),{'min': 0, 'max':500, 'palette':['#000000', '#023858', '#006837', '#1a9850', '#66bd63', '#a6d96a', '#d9ef8b','#ffffbf', '#fee08b', '#fdae61', '#f46d43', '#d73027']},'Water')
#Map.addLayer(LandCoverSelect)
#Map.addLayer(slope,{'min': 0, 'max':40, 'palette':['#140F13', '#084E82','#9CCEAA','#C5DC92','#E4D16A','#FFFEFA' ]},'Slope')
#Map.addLayer(ee.Image("users/iOne/litho_old"),{},'Lithology')
#Map.addLayer(CTI,{'min': 0, 'max':30, 'palette':['#2A2786','#578ECB','#B2D491', '#EF7F37', '#730B12']},'CTI')
#Map.addLayer(LithologyToCoeffs)
#Map.addLayer(lith_val)
###In landslide and PGV
##End dashboard

#Map.setCenter(142.9968, 50.5865, 12)
Map.setCenter(CenterLong, CenterLat, 9)
Map

Map(center=[46.95, 142.73], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(v…